# Table of Contents
1. [Data Cleaning](#data-cleaning)
2. [Custom One-Hot Encoding Function for Generated Data](#one-hot-enc)
3. [Running All Regression Models on Data with Train-Test Split](#initial-regression-model-run)
    1. [Compute RMSE Model List Function](#compute-rmse)
4. [Model Training Per ATM](#model-training-per-atm)
5. [Hyperparameter Tuning](#hyperparameter)
    1. [For SVR](#hyperparameter-svr)
    2. [For Lasso](#hyperparameter-lasso)
    3. [For Ridge](#hyperparameter-ridge)
    4. [For Elasticnet](#hyperparameter-enet)
    5. [For Bayesian Ridge](#hyperparameter-bayes)
    6. [For KNN](#hyperparameter-knn)
    7. [For Decision Tree](#hyperparameter-dtree)
    8. [For Random Forest](#hyperparameter-rforest)
6. [Polynomial Regression with Hyperparameter Tuning](#polynomial)
    

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime 

%matplotlib inline

In [5]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, BayesianRidge, ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.metrics import mean_squared_error, make_scorer, mean_absolute_percentage_error
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, validation_curve, cross_val_score, train_test_split

from sklearn.pipeline import Pipeline

# For measuring feature importance in tree based models like DTree and RandomForest
from sklearn.inspection import permutation_importance 
# For measuring feature importance in linear_models like Lasso, Ridge, etc.
from regressors.stats import coef_pval

<a name="data-cleaning"></a>
## Data Import and Cleaning 
<p> The function performs the following tasks, </p>
    <ol> 
        <li> Converts Weekday column to Uppercase because of format mismatch </li>
        <li> Converts Transaction Date column to Date Time object </li>
        <li> Removes the XYZ and Other Card Withdrawn Amounts columns target column (Total Amount Withdrawn) is the sum of these two columns </li>
        <li> Removes the Number of XYZ and Other Card Withdrawal columns because we will not know these values for test data</li>

In [6]:
data = pd.read_csv("../data/aggregated_atm_data.csv")

def data_import_and_cleaning(data):
    data['Weekday'] = data['Weekday'].str.upper()
    data['Transaction Date'] = pd.to_datetime(data['Transaction Date']) 
    new_data = data.drop(['Transaction Date', 'Amount withdrawn Other Card', 'Amount withdrawn XYZ Card', 
                          'No Of Withdrawals', 'No Of XYZ Card Withdrawals', 'No Of Other Card Withdrawals'], 
                         axis=1)
    
    return new_data

new_data = data_import_and_cleaning(data)

In [7]:
new_data.head()

,ATM Name,Total amount Withdrawn,Weekday,Festival Religion,Working Day,Holiday Sequence
0,Big Street ATM,123800,SATURDAY,H,H,WHH
1,Mount Road ATM,767900,SATURDAY,C,H,WHH
2,Airport ATM,503400,SATURDAY,C,H,WHH
3,KK Nagar ATM,945300,SATURDAY,C,H,WHH
4,Christ College ATM,287700,SATURDAY,C,H,WHH


## Breaking Transaction Date into Day, Month and Year

In [8]:
new_data['Month'] = data['Transaction Date'].dt.month
new_data['Day'] = data['Transaction Date'].dt.day
new_data['Year'] = data['Transaction Date'].dt.year

<a name="one-hot-enc"></a>
## Custom One-Hot Encoding Function for Generated Data

In [9]:
categorical_features_list = ['ATM Name', 'Weekday', 'Festival Religion', 'Working Day', 'Holiday Sequence', 
                             'Month', 'Day', 'Year']

In [10]:
# Convert all categorical columns to Dummy Data (One-Hot Encoding)
# drop_first = True to avoid the first column of each dummy column's result
# So if column = Gender and has two unique values Male and Female, get_dummies on this column creates two new columns
# male and female, if person male that column is 1 and the other is 0 and same for female column, but we only just need
# one of these columns, male or female, if male is 0 it guarantees person is female, for that reason drop_first=True
def convert_categorical_to_numerical(data, column_list):
    if 'ATM Name' in column_list:
        column_list.remove('ATM Name')
        temp_data = pd.get_dummies(data, columns=column_list , drop_first=True)
    
        # Do drop_first for all columns, except for ATM Name, because it becomes useful for accessing individual ATM test
        # data later on
        return pd.get_dummies(temp_data, columns=['ATM Name'])
    else:
        return pd.get_dummies(data, columns=column_list, drop_first=True)


# One-Hot Encoding and then one of the ATM Name columns has to be dropped because of the way the function is written
temp_numeric_data = convert_categorical_to_numerical(new_data, categorical_features_list)
numeric_data = temp_numeric_data.drop('ATM Name_KK Nagar ATM', axis=1)

In [11]:
numeric_data.head()

,Total amount Withdrawn,Weekday_MONDAY,Weekday_SATURDAY,Weekday_SUNDAY,Weekday_THURSDAY,Weekday_TUESDAY,Weekday_WEDNESDAY,Festival Religion_H,Festival Religion_M,Festival Religion_N,...,Year_2012,Year_2013,Year_2014,Year_2015,Year_2016,Year_2017,ATM Name_Airport ATM,ATM Name_Big Street ATM,ATM Name_Christ College ATM,ATM Name_Mount Road ATM
0,123800,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,767900,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,503400,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,945300,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,287700,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


<h3> Note that since all columns are non-numeric, scaling is not required</h3>

<a name="initial-regression-model-run"></a>
## Running Each Regression Model to get summary of all RMSEs
<b> One model for all ATMs </b>
<h3> Models tested </h3>
<ul>
    <li> KNN </li>
    <li> Linear Regression </li>
    <li> Ridge </li>
    <li> Elasticnet </li>
    <li> Lasso </li>
    <li> Bayesian Ridge </li>
    <li> Support Vector Regression </li>
    <li> Random Forest Regression </li>
    <li> Decision Tree Regression </li>
</ul>

In [12]:
models = {'K-Nearest Neighbours': KNeighborsRegressor(n_neighbors=36, n_jobs=-1), 'Vanilla Linear Regression': LinearRegression(), 
          'Ridge Linear Regression': Ridge(), 'Elasticnet Linear Regression': ElasticNet(), 
          'Lasso Linear Regression': Lasso(max_iter=1100), 'Bayesian Ridge Linear Regression': BayesianRidge(), 
          'Support Vector Regression': SVR(), 'Random Forest Regression': RandomForestRegressor(), 
          'Decision Tree Regression': DecisionTreeRegressor()}

atm_names = new_data['ATM Name'].unique()

<a name="compute-rmse"></a>
<h3> Compute RMSE function </h3>
<p> It loops over all models given, fits train data and predicts on test data </p>

In [13]:
def compute_rmse_model_list(models, X_train, y_train, X_test, y_test):
    print("RMSEs for each model")

    for model_name in models:
        model = models[model_name]
        model.fit(X_train, y_train)
        model_predictions = model.predict(X_test)
        model_rmse = np.sqrt(mean_squared_error(y_test, model_predictions))
        model_mape = mean_absolute_percentage_error(y_test, model_predictions)
        model_testing_score = model.score(X_test, y_test)
        model_training_score = model.score(X_train, y_train)
        print("For", model_name)
        print("\tTesting RMSE = {}".format(model_rmse))
        print("\tTesting MAPE = {}".format(model_mape))
        print("\tTraining Score =", model_training_score)
        print("\tTesting Score =", model_testing_score)

## Randomized Train Test Split, 70% Train, 30% Test

In [14]:
X = numeric_data.drop('Total amount Withdrawn', axis=1)
y = numeric_data['Total amount Withdrawn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)
compute_rmse_model_list(models, X_train, y_train, X_test, y_test)

RMSEs for each model
For K-Nearest Neighbours
	Testing RMSE = 265378.33194243954
	Testing MAPE = 7.783304302433827
	Training Score = 0.3819838460635303
	Testing Score = 0.34207226076275943
For Vanilla Linear Regression
	Testing RMSE = 251419.8192235889
	Testing MAPE = 8.425937522756863
	Training Score = 0.4013361571445506
	Testing Score = 0.40946411026285623
For Ridge Linear Regression
	Testing RMSE = 251395.20976856566
	Testing MAPE = 8.426500618763466
	Training Score = 0.401303262994798
	Testing Score = 0.4095797101802804
For Elasticnet Linear Regression
	Testing RMSE = 309411.6362690149
	Testing MAPE = 8.526314975060792
	Training Score = 0.10900893738529349
	Testing Score = 0.10562312336992907
For Lasso Linear Regression
	Testing RMSE = 251416.0051469024
	Testing MAPE = 8.426030449489193
	Training Score = 0.4013359781830569
	Testing Score = 0.40948202716449345
For Bayesian Ridge Linear Regression
	Testing RMSE = 251414.41701770766
	Testing MAPE = 8.422080318276114
	Training Score = 

<table align="left">
    <tr>
        <th> Model </th>
        <th> Test RMSE </th>
        <th> Train Score </th>
        <th> Test Score </th>
    </tr>
    <tr>
        <td> K-Nearest Neighbours </td>
        <td> 264286 </td>
        <td> 0.3768 </td>
        <td> 0.3474 </td>
    </tr>
    <tr>
        <td> Linear Regression </td>
        <td> 251419 </td>
        <td> 0.4013 </td>
        <td> 0.4094 </td>
    </tr>
    <tr>
        <td> Ridge Linear Regression </td>
        <td> 251395 </td>
        <td> 0.4013 </td>
        <td> 0.4095 </td>
    </tr>
    <tr>
        <td> Elasticnet Linear Regression </td>
        <td> 309411 </td>
        <td> 0.1090 </td>
        <td> 0.1056 </td>
    </tr>
    <tr>
        <td> Lasso Linear Regression </td>
        <td> 251416 </td>
        <td> 0.4013 </td>
        <td> 0.4094 </td>
    </tr>
    <tr>
        <td> Bayesian Ridge Linear Regression </td>
        <td> 251414 </td>
        <td> 0.4009 </td>
        <td> 0.4094 </td>
    </tr>
    <tr>
        <td> Support Vector Regression </td>
        <td> 330060 </td>
        <td> -0.0238 </td>
        <td> -0.0177 </td>
    </tr>
    <tr>
        <td> <b> Random Forest Regression </b> </td>
        <td> <b> 208204 </b> </td>
        <td> 0.9376 </td>
        <td> 0.5950 </td>
    </tr>
    <tr>
        <td> Decision Tree Regression </td>
        <td> 256902 </td>
        <td> 1.0 </td>
        <td> 0.3834 </td>
    </tr>  
</table>

<h3> After observing the testing and training scores, its clear that Random Forest Performed the Best and Decision Tree overfits, other than that all other algorithms perform not so great but there is no overfitting </h3>

<a name="model-training-per-atm"></a>
## Training all models for each ATM separately

In [10]:
def model_training_per_atm(atm_name):
    models = {'K-Nearest Neighbours': KNeighborsRegressor(n_neighbors=37, n_jobs=-1), 'Vanilla Linear Regression': LinearRegression(), 
          'Ridge Linear Regression': Ridge(), 'Elasticnet Linear Regression': ElasticNet(), 
          'Lasso Linear Regression': Lasso(max_iter=1100), 'Bayesian Ridge Linear Regression': BayesianRidge(), 
          'Support Vector Regression': SVR(), 'Random Forest Regression': RandomForestRegressor(), 
          'Decision Tree Regression': DecisionTreeRegressor()}
    
    categorical_features_list = ['Weekday', 'Festival Religion', 'Working Day', 'Holiday Sequence', 'Month', 'Day', 'Year']
    
    curr_atm_data = new_data[new_data['ATM Name'] == atm_name].drop('ATM Name', axis=1)
    numeric_curr_atm_data = convert_categorical_to_numerical(curr_atm_data, categorical_features_list)
    
    train_data = numeric_curr_atm_data[numeric_curr_atm_data['Year_2017'] == 0]
    test_data = numeric_curr_atm_data[numeric_curr_atm_data['Year_2017'] == 1]
    
    print("\nFor ATM:", atm_name)
    print("Number of training rows:",len(train_data))
    print("Number of testing rows:", len(test_data))
    print()

    X_train = train_data.drop('Total amount Withdrawn', axis=1)
    y_train = train_data['Total amount Withdrawn']

    X_test = test_data.drop('Total amount Withdrawn', axis=1)
    y_test = test_data['Total amount Withdrawn']
    
    compute_rmse_model_list(models, X_train, y_train, X_test, y_test)

In [13]:
for atm_name in new_data['ATM Name'].unique():
    model_training_per_atm(atm_name)
    
# Note: Bayesian Ridge Linear Regression does not converge for the Mount Road and KK Nagar models and increasing n_iter
# to even 5000 (default is 350) didn't change that at all


For ATM: Big Street ATM
Number of training rows: 2117
Number of testing rows: 237

RMSEs for each model
For K-Nearest Neighbours
	Testing RMSE = 181461.59767223545
	Training Score = 0.28999562819250424
	Testing Score = -0.36556725432459025
For Vanilla Linear Regression
	Testing RMSE = 276512.2844681913
	Training Score = 0.5444464813526175
	Testing Score = -2.1708266083172028
For Ridge Linear Regression
	Testing RMSE = 273587.69620255945
	Training Score = 0.5442115519020725
	Testing Score = -2.104107536431348
For Elasticnet Linear Regression
	Testing RMSE = 160736.02690753856
	Training Score = 0.13017936078590775
	Testing Score = -0.07144551642773656
For Lasso Linear Regression
	Testing RMSE = 276469.1320024553
	Training Score = 0.5444461398934355
	Testing Score = -2.169837008089656
For Bayesian Ridge Linear Regression
	Testing RMSE = 270701.1207086409
	Training Score = 0.5436357614309526
	Testing Score = -2.038951303826138
For Support Vector Regression
	Testing RMSE = 158495.819471549

c:\users\jasvin\appdata\local\programs\python\python38\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:474: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 318209236896.22656, tolerance: 12393046096.022762
  model = cd_fast.enet_coordinate_descent(


For Lasso Linear Regression
	Testing RMSE = 322203.5496961272
	Training Score = 0.5181017969667947
	Testing Score = -2.0088051198912362
For Bayesian Ridge Linear Regression
	Testing RMSE = 324014.4789707869
	Training Score = 0.5135989086530637
	Testing Score = -2.0427218501556643
For Support Vector Regression
	Testing RMSE = 322190.0963087881
	Training Score = -0.00016256288294114185
	Testing Score = -2.0085538639651888
For Random Forest Regression
	Testing RMSE = 330963.45681744564
	Training Score = 0.9219291450813367
	Testing Score = -2.174632850965621
For Decision Tree Regression
	Testing RMSE = 356024.4330009066
	Training Score = 1.0
	Testing Score = -2.6736097999720623

For ATM: Airport ATM
Number of training rows: 2058
Number of testing rows: 195

RMSEs for each model
For K-Nearest Neighbours
	Testing RMSE = 280998.75391993637
	Training Score = 0.21011877894017894
	Testing Score = -0.320081646974691
For Vanilla Linear Regression
	Testing RMSE = 270419.38396096433
	Training Score 

c:\users\jasvin\appdata\local\programs\python\python38\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:474: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4886918595366.969, tolerance: 40041415972.54263
  model = cd_fast.enet_coordinate_descent(


For Bayesian Ridge Linear Regression
	Testing RMSE = 449447.70286618423
	Training Score = 0.5109456907699065
	Testing Score = -0.46117843124706037
For Support Vector Regression
	Testing RMSE = 476342.10707451584
	Training Score = -0.010162086368770362
	Testing Score = -0.641280726160238
For Random Forest Regression
	Testing RMSE = 446472.98764000664
	Training Score = 0.9283288998089103
	Testing Score = -0.44190052404055336
For Decision Tree Regression
	Testing RMSE = 470676.8132168824
	Training Score = 1.0
	Testing Score = -0.6024723009525013

For ATM: Christ College ATM
Number of training rows: 2115
Number of testing rows: 240

RMSEs for each model
For K-Nearest Neighbours
	Testing RMSE = 454068.21878890437
	Training Score = 0.2887955781878394
	Testing Score = -0.48406666444966473
For Vanilla Linear Regression
	Testing RMSE = 563369.5987146889
	Training Score = 0.4884805976915243
	Testing Score = -1.2845360365183236
For Ridge Linear Regression
	Testing RMSE = 559635.1538683609
	Traini

c:\users\jasvin\appdata\local\programs\python\python38\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:474: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 303677188912.1094, tolerance: 13354579722.531445
  model = cd_fast.enet_coordinate_descent(


For Bayesian Ridge Linear Regression
	Testing RMSE = 554222.9690183143
	Training Score = 0.48620201491936166
	Testing Score = -1.2109567113667454
For Support Vector Regression
	Testing RMSE = 464427.1776615809
	Training Score = -0.01517727140145797
	Testing Score = -0.552553057061139
For Random Forest Regression
	Testing RMSE = 570587.0279230344
	Training Score = 0.9148511753369539
	Testing Score = -1.3434461927554369
For Decision Tree Regression
	Testing RMSE = 578248.1828044656
	Training Score = 1.0
	Testing Score = -1.4067986082933648


<table align="left">
    <tr>
        <th> ATM </th>
        <th> Best Algorithm </th>
        <th> Test RMSE </th>
        <th> Train Score </th>
        <th> Test Score </th>
    </tr>
    <tr>
        <td> Big Street ATM </td>
        <td> Support Vector Regression </td>
        <td> 158495 </td>
        <td> -0.0014 </td>
        <td> -0.0417 </td>
    </tr>
    <tr>
        <td> Mount Road ATM </td>
        <td> Linear Regression </td>
        <td> 322167 </td>
        <td> 0.5181 </td>
        <td> -2.0081 </td>
    </tr>
    <tr>
        <td> Airport ATM </td>
        <td> Ridge Linear Regression </td>
        <td> 270340 </td>
        <td> 0.3217 </td>
        <td> -0.2218 </td>
    </tr>
    <tr>
        <td> KK Nagar ATM </td>
        <td> Random Forest Regression </td>
        <td> 446472 </td>
        <td> 0.9283 </td>
        <td> -0.4419 </td>
    </tr>
    <tr>
        <td> Christ College ATM </td>
        <td> Elasticnet Linear Regression </td>
        <td> 437212 </td>
        <td> 0.1515 </td>
        <td> -0.3759 </td>
    </tr>
</table>

## Conclusions:
<ul>
    <li> Performance has drastically improved as a result of training on individual ATMs instead of a single model for all ATMs. </li>
    <li> There is no clear "best" algorithm which performs the best on all ATMs instead each ATM has its own best algorithm if Test RMSE is to be considered as the evaluation metric. </li>
    <li> The test RMSEs are also quite varied as KK Nagar, Christ College and Mount Road ATMs give quite high RMSEs as compared to the previous best of 2.7 lakhs, only Airport and Big Street give reasonable RMSEs </li>

<h3> Upto this point, removing outlier months from test set has had no effect, be it for the model with all ATMs or for individual ATMs, infact it made the RMSE worse at times, so the outlier month part is ignored for now </h3>

<a name="hyperparameter"></a>
## Hyperparameter Tuning (done for individual ATMs) with Train-Test Split with 10-fold Cross Validation

<p> The function below this cell takes the atm_name and the random search or grid search object and does all the tasks of converting to numeric data, train test split using 2017 data or regular train_test_split and then does Cross Validation and then computes the RMSE on the best estimator found after hyperparameter search </p>


## Also find Feature Importance for important features
<p> This is not doable for KNN and SVR </p>

<h3> 1. For Tree Models, i.e., Random Forest and Decision Tree </h3>

    Prints important features based on permutation_importance() function result
    And only displays those features whose importance is atleast twice more than the standard deviation of the 
    entire importance array

<h3> 2. For Linear Models, i.e., Elasticnet, Bayesian Ridge, Lasso, Ridge, etc. </h3>

    Prints p-values of all features (can be filtered down later)
    Significance level was set to 5%, so if p-value < 0.05, only then would a feature be considered as important

In [12]:
categorical_features_list = ['Weekday', 'Festival Religion', 'Working Day', 'Holiday Sequence', 'Month', 'Day', 'Year']

In [11]:
# Works for both RandomizedSearchCV() object and GridSearchCV() object
def model_training_hyperparam_per_atm(atm_name, new_data, categorical_features_list, 
                                      param_cv_obj, tree_model='Invalid'):
    
    curr_atm_data = new_data[new_data['ATM Name'] == atm_name].drop('ATM Name', axis=1)
    numeric_curr_atm_data = convert_categorical_to_numerical(curr_atm_data, categorical_features_list)
    
    X = numeric_curr_atm_data.drop('Total amount Withdrawn', axis=1)
    y = numeric_curr_atm_data['Total amount Withdrawn']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
    param_cv_obj.fit(X_train, y_train)
    print("Best Parameters:\n", param_cv_obj.best_params_)
    print("\nBest CV RMSE:", -param_cv_obj.best_score_)
    if 'mean_train_score' in param_cv_obj.cv_results_:
        # nanmean() is used here instead of mean() because for DecisionTree the mean_train_score actually contains nan
        # for some reason, so nanmean() ignores it while computing mean
        print("Best Training RMSE:", -np.nanmean(param_cv_obj.cv_results_['mean_train_score']))
    
    best_model = param_cv_obj.best_estimator_
    model_predictions = best_model.predict(X_test)
    model_rmse = np.sqrt(mean_squared_error(y_test, model_predictions))
    
    print("Test RMSE:", model_rmse)
    
    column_names = X_train.columns
    
    if tree_model == False:
        # p_val_array[1:] because 0-th index is reserved for intercept p-val which is not required for us
        p_val_array = coef_pval(best_model, X_train, y_train)
        [print("{} : {}".format(x, y)) for x, y in zip(column_names, p_val_array[1:]) if y < 0.05]
        
    elif tree_model == True:
        r = permutation_importance(best_model, X_test, y_test, n_repeats=30, random_state=0)
        for i in r.importances_mean.argsort()[::-1]:
            if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
                print(f"{column_names[i]:<8}: "
                      f"{r.importances_mean[i]:.3f}"
                      f" +/- {r.importances_std[i]:.3f}")
        
    print("------------------------------------------------------------------------\n")    

<a name="hyperparameter-svr"></a>
## SVR Hyperparameter Tuning
<b> Parameters tuned, C and epsilon </b>

<p> <b> Doing only Random Search because Grid Search takes a very very long time </b> </p>

In [17]:
svr_param_grid = {'C': list(np.logspace(-3, 5, num=25)),
                         'epsilon': list(np.logspace(-3, 3, num=25))}

random_cv_svr = RandomizedSearchCV(SVR(), svr_param_grid, n_iter=50, cv=10, verbose=2, 
                                   n_jobs=-1, scoring='neg_root_mean_squared_error', return_train_score=True)

for atm_name in atm_names:
    print("For ATM:", atm_name)
    model_training_hyperparam_per_atm(atm_name, new_data, categorical_features_list, random_cv_svr)

For ATM: Big Street ATM
Fitting 10 folds for each of 50 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   24.7s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:   56.6s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.3min finished


Best Parameters:
 {'epsilon': 31.622776601683793, 'C': 100000.0}

Best CV RMSE: 117078.97246935354
Best Training RMSE: 152787.39282150136
Test RMSE: 110942.75941664934
------------------------------------------------------------------------

For ATM: Mount Road ATM
Fitting 10 folds for each of 50 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   21.7s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:   49.6s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.1min finished


Best Parameters:
 {'epsilon': 0.0017782794100389228, 'C': 100000.0}

Best CV RMSE: 184928.2336764616
Best Training RMSE: 245904.6915620005
Test RMSE: 172948.16892758766
------------------------------------------------------------------------

For ATM: Airport ATM
Fitting 10 folds for each of 50 candidates, totalling 500 fits


...

Best Parameters:
 {'epsilon': 177.82794100389228, 'C': 100000.0}

Best CV RMSE: 174370.4068236465
Best Training RMSE: 196896.4981433494
Test RMSE: 175196.22234095153
------------------------------------------------------------------------

For ATM: KK Nagar ATM
Fitting 10 folds for each of 50 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   22.8s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:   50.4s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.2min finished


Best Parameters:
 {'epsilon': 0.5623413251903491, 'C': 21544.346900318822}

Best CV RMSE: 353724.97533654666
Best Training RMSE: 434567.0656866024
Test RMSE: 354997.56093387585
------------------------------------------------------------------------

For ATM: Christ College ATM
Fitting 10 folds for each of 50 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   20.9s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:   48.2s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.1min finished


Best Parameters:
 {'epsilon': 0.1, 'C': 100000.0}

Best CV RMSE: 218385.86763734784
Best Training RMSE: 273135.22250586585
Test RMSE: 203072.03789112024
------------------------------------------------------------------------



<table align="left">
    <tr>
        <th> ATM </th>
        <th> Train RMSE </th>
        <th> CV RMSE </th>
        <th> Test RMSE </th>
    </tr>
    <tr>
        <td> Big Street ATM </td>
        <td> 152787 </td>
        <td> 117078 </td>
        <td> 110942 </td>
    </tr>
    <tr>
        <td> Mount Road ATM </td>
        <td> 245904 </td>
        <td> 184928 </td>
        <td> 172948 </td>
    </tr>
    <tr>
        <td> Airport ATM </td>
        <td> 196896 </td>
        <td> 174370 </td>
        <td> 175196 </td>
    </tr>
    <tr>
        <td> KK Nagar ATM </td>
        <td> 434567 </td>
        <td> 353724 </td>
        <td> 354997 </td>
    </tr>
    <tr>
        <td> Christ College ATM </td>
        <td> 273135 </td>
        <td> 218385 </td>
        <td> 203072 </td>
    </tr>
</table>

<a name="hyperparameter-lasso"></a>
## Hyperparameter Tuning for Lasso
<b> Alpha is the only tunable parameter here </b>

In [19]:
lasso_param_grid = {'alpha':np.logspace(-3, 3, num=50)}
grid_cv_lasso = GridSearchCV(Lasso(max_iter=2000), lasso_param_grid, verbose=2, n_jobs=-1, 
                             scoring='neg_root_mean_squared_error', cv=10, return_train_score=True)

for atm_name in atm_names:
    print("\nFor", atm_name)
    model_training_hyperparam_per_atm(atm_name, new_data, categorical_features_list, 
                                      grid_cv_lasso, tree_model=False)


For Big Street ATM
Fitting 10 folds for each of 50 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   16.4s
[Parallel(n_jobs=-1)]: Done 358 tasks      | elapsed:   26.7s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   27.8s finished


Best Parameters:
 {'alpha': 323.745754281764}

Best CV RMSE: 115439.58205293913
Best Training RMSE: 111188.4594907152
Test RMSE: 113291.11172135144
Month_10 : 0.026837121146669363
Month_12 : 0.004204576685199246
Day_6 : 2.929352309344324e-05
Day_8 : 0.002935600260723703
Day_9 : 0.00022062193685967735
Day_11 : 0.00033683977118204034
Day_12 : 1.7640814217179113e-05
Day_31 : 0.03762929162671025
Year_2012 : 5.329070518200751e-15
Year_2013 : 0.0
Year_2014 : 0.0
Year_2015 : 0.0
Year_2016 : 0.0
Year_2017 : 0.0
------------------------------------------------------------------------


For Mount Road ATM
Fitting 10 folds for each of 50 candidates, totalling 500 fits

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   13.9s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:   25.2s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   26.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.



Best Parameters:
 {'alpha': 44.98432668969444}

Best CV RMSE: 178687.2089742815
Best Training RMSE: 170956.18121349913
Test RMSE: 179497.18959146913
Weekday_MONDAY : 0.038391167839486595
Weekday_SATURDAY : 6.870635615996434e-09
Weekday_THURSDAY : 0.004829260299984206
Weekday_TUESDAY : 0.046639527183068186
Weekday_WEDNESDAY : 0.0004357559652794496
Working Day_W : 0.025085983662012534
Month_3 : 0.03759145155742449
Month_4 : 0.018799665608438376
Month_5 : 1.8784389252068934e-05
Month_6 : 0.015869324383895433
Month_8 : 0.009096804997042351
Month_9 : 0.03176125876204394
Month_10 : 0.0113916358342383
Month_11 : 0.0021135383290171728
Day_5 : 0.03125218549386011
Day_10 : 0.00030167000055758564
Day_13 : 7.255526739857743e-06
Day_14 : 2.1116016430955398e-06
Day_15 : 3.74298370076076e-11
Day_16 : 8.036675769318435e-10
Day_17 : 4.287333821295647e-09
Day_18 : 2.1917787362824015e-08
Day_19 : 3.4989327033496664e-05
Day_20 : 6.727563184760754e-06
Day_21 : 6.057119250613141e-10
Day_22 : 8.600454792784

...

[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   16.4s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:   29.9s
[Parallel(n_jobs=-1)]: Done 493 out of 500 | elapsed:   32.0s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   32.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Best Parameters:
 {'alpha': 1000.0}

Best CV RMSE: 313189.34459445137
Best Training RMSE: 300542.5505786474
Test RMSE: 341981.8797105739
Weekday_SUNDAY : 0.005647919591432515
Festival Religion_H : 0.019874561974367433
Month_11 : 0.02105705878183395
Month_12 : 0.008888007788053764
Day_4 : 0.006207220578884343
Day_6 : 1.8214385555381796e-10
Day_7 : 0.0034920144808363585
Day_10 : 0.001158332058677214
Day_24 : 0.004359224387390892
Day_27 : 0.04653854414815184
Year_2012 : 0.0
Year_2013 : 0.0
Year_2014 : 0.0
Year_2015 : 4.246514251349254e-11
Year_2016 : 4.131837094689672e-10
Year_2017 : 2.220446049250313e-16
------------------------------------------------------------------------


For Christ College ATM
Fitting 10 folds for each of 50 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   14.3s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:   25.6s


Best Parameters:
 {'alpha': 79.06043210907701}

Best CV RMSE: 208506.62853769763
Best Training RMSE: 200744.37831592732
Test RMSE: 204298.34256045986
Weekday_SATURDAY : 8.925062482401103e-07
Weekday_THURSDAY : 0.04497853521227135
Weekday_WEDNESDAY : 0.017121295137891623
Working Day_W : 0.0003102233389082887
Holiday Sequence_HHW : 0.030857140501935554
Holiday Sequence_WHH : 0.04211190285355504
Holiday Sequence_WHW : 0.018084666833941165
Month_7 : 0.04559345501287959
Month_11 : 0.0004190963555965599
Month_12 : 0.0002430848936845198
Day_7 : 0.027216083029303784
Day_8 : 0.030515804807257796
Day_9 : 0.0001638306559188507
Day_10 : 1.7126346477214227e-05
Day_14 : 0.001424308300719268
Day_16 : 8.13315743803944e-05
Day_17 : 0.012978797522592567
Day_18 : 0.0002483764805449873
Day_19 : 0.0015101872161811247
Day_20 : 0.0021863290086976406
Day_21 : 2.1451885238610302e-05
Day_22 : 2.9172596240689685e-05
Day_23 : 1.523509860490435e-07
Day_24 : 8.805738360706528e-09
Day_25 : 3.219974188617414e-06
Day_

[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   27.2s finished


<table align="left">
    <tr>
        <th> ATM </th>
        <th> Train RMSE </th>
        <th> CV RMSE </th>
        <th> Test RMSE </th>
    </tr>
    <tr>
        <td> Big Street ATM </td>
        <td> 111188 </td>
        <td> 115439 </td>
        <td> 113291 </td>
    </tr>
    <tr>
        <td> Mount Road ATM </td>
        <td> 170956 </td>
        <td> 178687 </td>
        <td> 179497 </td>
    </tr>
    <tr>
        <td> Airport ATM </td>
        <td> 164569 </td>
        <td> 171317 </td>
        <td> 179650 </td>
    </tr>
    <tr>
        <td> KK Nagar ATM </td>
        <td> 300542 </td>
        <td> 313189 </td>
        <td> 341981 </td>
    </tr>
    <tr>
        <td> Christ College ATM </td>
        <td> 200744 </td>
        <td> 208506 </td>
        <td> 204298 </td>
    </tr>
</table>

<a name="hyperparameter-ridge"></a>
## Hyperparameter Tuning for Ridge
<b> Alpha is again the only tunable parameter here </b>

In [22]:
ridge_param_grid = {'alpha':np.logspace(-3, 3, num=50)}
grid_cv_ridge = GridSearchCV(Ridge(), ridge_param_grid, verbose=5, n_jobs=-1, scoring='neg_root_mean_squared_error'
                            ,return_train_score=True)

for atm_name in atm_names:
    print("\nFor", atm_name)
    model_training_hyperparam_per_atm(atm_name, new_data, categorical_features_list, 
                                      grid_cv_ridge, tree_model=False)    


For Big Street ATM
Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:    0.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.0s


Best Parameters:
 {'alpha': 1.5264179671752334}

Best CV RMSE: 117935.23274571684
Best Training RMSE: 120326.63005324436
Test RMSE: 107950.65157317226
Weekday_SATURDAY : 0.0019255040878563001
Weekday_THURSDAY : 0.005441490658050885
Weekday_TUESDAY : 0.0003432222142065733
Weekday_WEDNESDAY : 0.009764274144803187
Festival Religion_N : 0.02050332220477058
Holiday Sequence_HHW : 0.018625236067314255
Holiday Sequence_WHW : 0.009977674649686463
Month_9 : 0.007779936370949203
Month_10 : 0.0017335434558145568
Month_12 : 0.003020051273987967
Day_6 : 5.635946424220606e-06
Day_8 : 5.7910894911694655e-05
Day_9 : 1.1418972579946285e-05
Day_11 : 0.0019256872223072374
Day_12 : 0.0010461591957919847
Day_17 : 0.004091033396562915
Day_19 : 0.03453727952574437
Day_23 : 0.014440801812289239
Day_24 : 0.0037673924533676573
Day_25 : 0.011093452306050189
Day_27 : 0.029360069667329736
Day_31 : 0.035154992587666634
Year_2012 : 0.0
Year_2013 : 0.0
Year_2014 : 0.0
Year_2015 : 0.0
Year_2016 : 0.0
Year_2017 : 0.0
-

[Parallel(n_jobs=-1)]: Done 236 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:    0.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Best Parameters:
 {'alpha': 0.49417133613238334}

Best CV RMSE: 178445.2138392486
Best Training RMSE: 180400.03880122502
Test RMSE: 182349.69269470643
Weekday_SATURDAY : 5.976634080973042e-07
Month_2 : 0.002588927826562948
Month_3 : 0.003029719510572848
Month_4 : 0.0007585533726812699
Month_5 : 6.462929363015846e-06
Month_6 : 0.005812571826488755
Month_7 : 0.009214509862349729
Month_8 : 0.0051949263065940166
Month_9 : 0.006030288947430185
Month_10 : 0.015517771682045378
Month_11 : 0.0012402022808886137
Month_12 : 0.04440314872576878
Day_10 : 1.1087848234003772e-05
Day_13 : 0.0001999181988581178
Day_14 : 1.5686166432438142e-07
Day_15 : 2.9508300247726993e-09
Day_16 : 5.481797238360286e-10
Day_17 : 3.60037340727537e-05
Day_18 : 5.797132835994034e-07
Day_19 : 6.354993125068376e-07
Day_20 : 3.4550124721199893e-06
Day_21 : 3.3517433273289043e-09
Day_22 : 2.095856821426878e-11
Day_23 : 9.273914969298858e-12
Day_24 : 2.535415013493747e-07
Day_25 : 2.60840545607266e-07
Day_26 : 1.8222090503172

[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:    0.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.0s


Best Parameters:
 {'alpha': 14.563484775012444}

Best CV RMSE: 177446.78922084943
Best Training RMSE: 172948.0539562998
Test RMSE: 168899.60492751581
Month_3 : 0.020514641884954887
Day_6 : 0.04807798114195094
Day_8 : 0.0025765350780135865
Day_17 : 0.040893296891456155
Day_24 : 0.03518572073715487
Year_2012 : 4.743268888773855e-09
Year_2013 : 2.220446049250313e-16
Year_2014 : 5.245718897928242e-06
Year_2016 : 0.0
Year_2017 : 3.643920720719507e-10
------------------------------------------------------------------------


For KK Nagar ATM
Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:    0.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.0s


Best Parameters:
 {'alpha': 6.25055192527397}

Best CV RMSE: 321567.8501408209
Best Training RMSE: 319819.20601955464
Test RMSE: 327358.86961965245
Weekday_SUNDAY : 0.008838963121154952
Weekday_THURSDAY : 0.04229494843367321
Weekday_WEDNESDAY : 0.0009947555940106323
Festival Religion_H : 0.033166024261570115
Month_7 : 0.0194804695711579
Month_8 : 0.037136068789373144
Month_9 : 0.0009921283659906077
Month_11 : 9.937531668091637e-05
Month_12 : 3.043468878249911e-06
Day_4 : 0.009332562966821722
Day_6 : 8.028000963999915e-06
Day_7 : 0.022629873557510782
Day_10 : 0.0019855269660331576
Day_16 : 0.022180178011782292
Day_18 : 0.0024682648887968117
Day_19 : 0.002611729423841025
Day_20 : 0.03699405720190496
Day_21 : 0.044902757008916216
Day_22 : 0.003356431227035772
Day_23 : 0.00018263433276355379
Day_24 : 4.44611197736311e-05
Day_27 : 0.02544351907150766
Year_2012 : 0.0
Year_2013 : 0.0
Year_2014 : 0.0
Year_2015 : 0.0
Year_2016 : 3.777231860624397e-10
Year_2017 : 9.769962616701378e-15
----------

[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:    0.8s finished


<table align="left">
    <tr>
        <th> ATM </th>
        <th> Train RMSE </th>
        <th> CV RMSE </th>
        <th> Test RMSE </th>
    </tr>
    <tr>
        <td> Big Street ATM </td>
        <td> 120326 </td>
        <td> 117935 </td>
        <td> 107950 </td>
    </tr>
    <tr>
        <td> Mount Road ATM </td>
        <td> 180400 </td>
        <td> 178445 </td>
        <td> 182349 </td>
    </tr>
    <tr>
        <td> Airport ATM </td>
        <td> 172948 </td>
        <td> 177446 </td>
        <td> 168899 </td>
    </tr>
    <tr>
        <td> KK Nagar ATM </td>
        <td> 319819 </td>
        <td> 321567 </td>
        <td> 327358 </td>
    </tr>
    <tr>
        <td> Christ College ATM </td>
        <td> 210913 </td>
        <td> 210213 </td>
        <td> 202712 </td>
    </tr>
</table>

<h3> Relatively worse results than Lasso, also often has CV RMSEs lower than Training RMSEs </h3>

<a name="hyperparameter-enet"></a>
## Hyperparameter tuning for Elasticnet
<b> Parameters tuned, l1_ratio and alpha </b>

In [23]:
elastic_param_grid = {'l1_ratio': [.1, .5, .7, .9, .95, .99, 1], 'alpha': np.logspace(-3, 3, num=50)}
                      
grid_cv_elastic = GridSearchCV(ElasticNet(), elastic_param_grid, n_jobs=-1,
                                       scoring='neg_root_mean_squared_error', verbose=2, cv=10, return_train_score=True)

for atm_name in atm_names:
    print("\nFor", atm_name)
    model_training_hyperparam_per_atm(atm_name, new_data, categorical_features_list, 
                                      grid_cv_elastic, tree_model=False)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.



For Big Street ATM
Fitting 10 folds for each of 350 candidates, totalling 3500 fits


[Parallel(n_jobs=-1)]: Done  58 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 300 tasks      | elapsed:   13.8s
[Parallel(n_jobs=-1)]: Done 706 tasks      | elapsed:   28.0s
[Parallel(n_jobs=-1)]: Done 1520 tasks      | elapsed:   41.7s
[Parallel(n_jobs=-1)]: Done 3500 out of 3500 | elapsed:   53.5s finished


Best Parameters:
 {'alpha': 0.016768329368110083, 'l1_ratio': 0.95}

Best CV RMSE: 114146.27041476921
Best Training RMSE: 131044.92562240835
Test RMSE: 114079.44083638032
Weekday_MONDAY : 0.007935497624020016
Weekday_SATURDAY : 0.02126410049094596
Weekday_SUNDAY : 0.036483947066974665
Weekday_THURSDAY : 0.0005448645188794732
Weekday_TUESDAY : 0.000108254578542466
Weekday_WEDNESDAY : 0.00012778111037015094
Festival Religion_M : 0.028431867577812575
Festival Religion_N : 0.015405532512669406
Holiday Sequence_WHW : 0.007327172433438722
Month_9 : 0.04981956493270112
Month_10 : 0.002804583024861529
Month_12 : 0.005851401350503993
Day_6 : 0.0012609602841469059
Day_8 : 0.00943888407855753
Day_9 : 0.00012083073714985737
Day_11 : 5.737755549084511e-05
Day_12 : 0.00026211776245621543
Day_17 : 0.0007893788755022335
Day_19 : 0.028838734386153586
Day_21 : 0.04470207668928472
Day_22 : 0.03598474611131941
Day_23 : 0.0017296723169926143
Day_24 : 0.0008820447372788198
Day_25 : 0.0006550626164745221
Day

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done 414 tasks      | elapsed:   17.9s
[Parallel(n_jobs=-1)]: Done 1076 tasks      | elapsed:   35.0s
[Parallel(n_jobs=-1)]: Done 2764 tasks      | elapsed:   50.0s
[Parallel(n_jobs=-1)]: Done 3500 out of 3500 | elapsed:   52.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


...

[Parallel(n_jobs=-1)]: Done  58 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 300 tasks      | elapsed:   13.6s
[Parallel(n_jobs=-1)]: Done 706 tasks      | elapsed:   27.8s
[Parallel(n_jobs=-1)]: Done 1584 tasks      | elapsed:   42.3s
[Parallel(n_jobs=-1)]: Done 3500 out of 3500 | elapsed:   54.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Best Parameters:
 {'alpha': 0.012648552168552958, 'l1_ratio': 0.7}

Best CV RMSE: 319457.99141757085
Best Training RMSE: 358609.6183091785
Test RMSE: 325347.27314054396
Weekday_SATURDAY : 0.024149053589428204
Weekday_SUNDAY : 0.006487644534830528
Weekday_WEDNESDAY : 0.0013471905378208415
Festival Religion_H : 0.01921479914275248
Month_7 : 0.0013259950113628793
Month_9 : 0.01666990512008848
Month_11 : 0.0002770763780048924
Month_12 : 6.792533859267813e-05
Day_4 : 0.04360020052914937
Day_6 : 1.97445837768484e-06
Day_10 : 0.01913622994849029
Day_16 : 0.012752970550160114
Day_18 : 0.006754297731055292
Day_19 : 0.0032666263816722996
Day_20 : 0.03930525812920882
Day_21 : 0.0043153387585572744
Day_22 : 0.0029570741212285867
Day_23 : 0.00286932213520652
Day_24 : 5.3703296603480055e-05
Day_25 : 0.016264830986760348
Day_27 : 0.0014626230233525384
Year_2012 : 0.0
Year_2013 : 0.0
Year_2014 : 0.0
Year_2015 : 0.0
Year_2016 : 1.751621070411602e-11
Year_2017 : 0.0
-------------------------------------

[Parallel(n_jobs=-1)]: Done  58 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 300 tasks      | elapsed:   13.6s
[Parallel(n_jobs=-1)]: Done 706 tasks      | elapsed:   27.9s
[Parallel(n_jobs=-1)]: Done 1656 tasks      | elapsed:   43.5s


Best Parameters:
 {'alpha': 59.636233165946365, 'l1_ratio': 1}

Best CV RMSE: 209106.93881861915
Best Training RMSE: 234834.47527783667
Test RMSE: 204999.30813885073
Weekday_SATURDAY : 2.016198319410023e-10
Weekday_THURSDAY : 0.003930932188116243
Weekday_WEDNESDAY : 0.0006952300794276223
Working Day_W : 0.0018840153702910367
Month_11 : 0.02818867593109542
Month_12 : 2.7763846426331895e-06
Day_3 : 0.024983917525233323
Day_9 : 8.571733518469848e-05
Day_10 : 0.004580698685753504
Day_13 : 0.0044248158717383745
Day_14 : 8.165872047172229e-05
Day_15 : 0.01774858344576158
Day_16 : 5.325840032099904e-06
Day_17 : 0.0036176388992186226
Day_18 : 4.175190548849628e-08
Day_19 : 0.00034152636480722087
Day_20 : 0.0006418583855867244
Day_21 : 6.7991748524676154e-06
Day_22 : 5.781176750652151e-07
Day_23 : 9.819014046286156e-09
Day_24 : 4.63519156390646e-09
Day_25 : 3.5719145875967584e-05
Day_26 : 5.8116771395333444e-09
Day_27 : 3.4876087811053225e-06
Day_28 : 5.9970266359510305e-06
Day_29 : 6.083605547

[Parallel(n_jobs=-1)]: Done 3500 out of 3500 | elapsed:   55.9s finished


<table align="left">
    <tr>
        <th> ATM </th>
        <th> Train RMSE </th>
        <th> CV RMSE </th>
        <th> Test RMSE </th>
    </tr>
    <tr>
        <td> Big Street ATM </td>
        <td> 131044 </td>
        <td> 114146 </td>
        <td> 114079 </td>
    </tr>
    <tr>
        <td> Mount Road ATM </td>
        <td> 180400 </td>
        <td> 178445 </td>
        <td> 182349 </td>
    </tr>
    <tr>
        <td> Airport ATM </td>
        <td> 181845 </td>
        <td> 174762 </td>
        <td> 172143 </td>
    </tr>
    <tr>
        <td> KK Nagar ATM </td>
        <td> 358609 </td>
        <td> 319457 </td>
        <td> 325347 </td>
    </tr>
    <tr>
        <td> Christ College ATM </td>
        <td> 234834 </td>
        <td> 209106 </td>
        <td> 204999 </td>
    </tr>
</table>

<h3> A bit worse than Lasso and Ridge both, also shows the same behaviour of lower CV RMSEs than training </h3>

<a name="hyperparameter-bayes"></a>

## Hyperparameter Tuning for Bayesian Ridge Regression

<b> Parameters tuned, alpha_1, alpha_2, lambda_1, lambda_2 </b>

In [24]:
bayes_param_grid = {'alpha_1': np.logspace(-3, 3, num=50), 'alpha_2': np.logspace(-3, 3, num=50),
                   'lambda_1': np.logspace(-3, 3, num=50), 'lambda_2': np.logspace(-3, 3, num=50)}
                      
random_cv_bayes = RandomizedSearchCV(BayesianRidge(), bayes_param_grid, n_iter=200, n_jobs=-1, 
                                      scoring='neg_root_mean_squared_error', verbose=2, cv=10, return_train_score=True)

for atm_name in atm_names:
    print("\nFor", atm_name)
    model_training_hyperparam_per_atm(atm_name, new_data, categorical_features_list, 
                                      random_cv_bayes, tree_model=False)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.



For Big Street ATM
Fitting 10 folds for each of 200 candidates, totalling 2000 fits


[Parallel(n_jobs=-1)]: Done 128 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 1096 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:   13.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Best Parameters:
 {'lambda_2': 3.5564803062231287, 'lambda_1': 0.005428675439323859, 'alpha_2': 0.0017575106248547913, 'alpha_1': 244.205309454865}

Best CV RMSE: 115811.80090594175
Best Training RMSE: 151847.10720056025
Test RMSE: 111834.65827364006
Weekday_MONDAY : 0.04202731758688705
Weekday_THURSDAY : 0.009171588906860872
Weekday_TUESDAY : 0.0007467179513971356
Weekday_WEDNESDAY : 0.0057123747323946095
Festival Religion_M : 0.049748996776970866
Festival Religion_N : 0.006377634983613323
Month_2 : 0.0458166615323512
Month_9 : 0.03309434128920352
Month_10 : 0.015302152625542575
Month_12 : 0.004373186551756669
Day_6 : 0.00044235767927003344
Day_8 : 0.020122268974851698
Day_9 : 0.00011395784790368957
Day_10 : 0.011407799937356966
Day_11 : 0.04016256349735392
Day_12 : 0.001206781384754052
Day_17 : 0.0036170760065674656
Day_19 : 0.0236236066342419
Day_21 : 0.018946815242651738
Day_22 : 0.010298473243874673
Day_23 : 0.001013585686594931
Day_24 : 7.089228904266776e-05
Day_25 : 0.0002057536

[Parallel(n_jobs=-1)]: Done 128 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 1096 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:   12.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


...

[Parallel(n_jobs=-1)]: Done 128 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 1096 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:   13.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Best Parameters:
 {'lambda_2': 754.3120063354608, 'lambda_1': 0.009540954763499945, 'alpha_2': 1.5264179671752334, 'alpha_1': 0.001}

Best CV RMSE: 323776.6113030715
Best Training RMSE: 426985.01097276434
Test RMSE: 315454.8068249778
Weekday_SATURDAY : 0.011463198784667394
Weekday_SUNDAY : 0.004107927281259638
Weekday_THURSDAY : 0.02494075201653634
Weekday_WEDNESDAY : 0.00010848369761307097
Festival Religion_H : 0.03051989602841365
Month_9 : 0.011194583806019676
Month_11 : 0.001293860149171877
Day_6 : 2.6250054720788896e-05
Day_10 : 0.011141016074434518
Day_16 : 0.010622821569122198
Day_17 : 0.0313112180984525
Day_18 : 0.000986062393949716
Day_19 : 0.02001502957018353
Day_20 : 0.005459386965305679
Day_22 : 0.0027752769415447798
Day_23 : 0.0010485699449220842
Day_24 : 2.419897792815817e-06
Day_25 : 0.0026053019473506467
Day_27 : 0.0012827159206350025
Day_31 : 0.006009908410387066
Year_2012 : 0.0
Year_2013 : 0.0
Year_2014 : 0.0
Year_2015 : 1.4654943925052066e-14
Year_2016 : 1.75940817470

[Parallel(n_jobs=-1)]: Done 128 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 1096 tasks      | elapsed:    7.2s


Best Parameters:
 {'lambda_2': 10.985411419875572, 'lambda_1': 0.016768329368110083, 'alpha_2': 0.012648552168552958, 'alpha_1': 1000.0}

Best CV RMSE: 208420.8298252251
Best Training RMSE: 267322.1627054598
Test RMSE: 207321.76924605123
Weekday_SATURDAY : 3.3839150193060163e-07
Weekday_THURSDAY : 0.0439303250428158
Weekday_WEDNESDAY : 0.03142288352535938
Working Day_W : 0.0008160254348132945
Holiday Sequence_WHH : 0.0037664108713622024
Holiday Sequence_WHW : 0.016513655698416008
Month_7 : 0.036814737463673364
Month_11 : 0.0034816454622859094
Month_12 : 6.003164915613368e-06
Day_3 : 0.009852136483808449
Day_7 : 0.026976995031774642
Day_8 : 0.04477723227980013
Day_9 : 5.5779492520002805e-05
Day_10 : 0.00848112720142824
Day_11 : 0.012628594360414747
Day_14 : 0.0019368863068787157
Day_16 : 7.79908423220732e-05
Day_17 : 4.760603686038145e-05
Day_18 : 2.416041482433684e-07
Day_19 : 0.003510002192397721
Day_20 : 0.0037958554061705563
Day_21 : 9.376573749886319e-05
Day_22 : 5.802438821200795e

[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:   13.1s finished


<table align="left">
    <tr>
        <th> ATM </th>
        <th> Train RMSE </th>
        <th> CV RMSE </th>
        <th> Test RMSE </th>
    </tr>
    <tr>
        <td> Big Street ATM </td>
        <td> 151847 </td>
        <td> 115811 </td>
        <td> 111834 </td>
    </tr>
    <tr>
        <td> Mount Road ATM </td>
        <td> 253171 </td>
        <td> 177595 </td>
        <td> 184030 </td>
    </tr>
    <tr>
        <td> Airport ATM </td>
        <td> 202133 </td>
        <td> 174285 </td>
        <td> 172490 </td>
    </tr>
    <tr>
        <td> KK Nagar ATM </td>
        <td> 426985 </td>
        <td> 323776 </td>
        <td> 315454 </td>
    </tr>
    <tr>
        <td> Christ College ATM </td>
        <td> 267322 </td>
        <td> 208420 </td>
        <td> 207321 </td>
    </tr>
</table>

<h3> Again Training RMSEs are higher than CV RMSEs, but still performance is good, although Elasticnet performs marginally better and Lasso performs way better, so not much point in using this </h3>

<a name="hyperparameter-knn"></a>
## Hyperparameter Tuning for KNN
<b> n_neighbors, leaf_size and p (distance metric selection) are the tunable parameters here </b>

In [25]:
knn_param_grid = {'n_neighbors': list(range(1,50)), 'leaf_size': list(range(1,50)), 'p': [1, 2]}
                      
random_cv_knn = RandomizedSearchCV(KNeighborsRegressor(), knn_param_grid, n_iter=25, n_jobs=-1, 
                                      scoring='neg_root_mean_squared_error', verbose=2, cv=10, return_train_score=True)

for atm_name in atm_names:
    print("\nFor", atm_name)
    model_training_hyperparam_per_atm(atm_name, new_data, categorical_features_list, random_cv_knn)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.



For Big Street ATM
Fitting 10 folds for each of 25 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   25.2s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   40.9s finished


Best Parameters:
 {'p': 2, 'n_neighbors': 47, 'leaf_size': 13}

Best CV RMSE: 137223.93079557377
Best Training RMSE: 135572.51080622335
Test RMSE: 134697.4374030493
------------------------------------------------------------------------


For Mount Road ATM
Fitting 10 folds for each of 25 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   16.9s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   30.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   15.6s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   28.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Best Parameters:
 {'p': 2, 'n_neighbors': 24, 'leaf_size': 24}

Best CV RMSE: 344799.9406417206
Best Training RMSE: 317543.31087899185
Test RMSE: 355962.5758796952
------------------------------------------------------------------------


For Christ College ATM
Fitting 10 folds for each of 25 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   15.8s


Best Parameters:
 {'p': 2, 'n_neighbors': 44, 'leaf_size': 28}

Best CV RMSE: 237546.83653303134
Best Training RMSE: 216635.30965126475
Test RMSE: 235123.5393551469
------------------------------------------------------------------------



[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   27.2s finished


<table align="left">
    <tr>
        <th> ATM </th>
        <th> Train RMSE </th>
        <th> CV RMSE </th>
        <th> Test RMSE </th>
    </tr>
    <tr>
        <td> Big Street ATM </td>
        <td> 135572 </td>
        <td> 137223 </td>
        <td> 134697 </td>
    </tr>
    <tr>
        <td> Mount Road ATM </td>
        <td> 186913 </td>
        <td> 207823 </td>
        <td> 212238 </td>
    </tr>
    <tr>
        <td> Airport ATM </td>
        <td> 171969 </td>
        <td> 182027 </td>
        <td> 187217 </td>
    </tr>
    <tr>
        <td> KK Nagar ATM </td>
        <td> 317543 </td>
        <td> 344799 </td>
        <td> 355962 </td>
    </tr>
    <tr>
        <td> Christ College ATM </td>
        <td> 216635 </td>
        <td> 237546 </td>
        <td> 235123 </td>
    </tr>
</table>

<h3> Not as good CV RMSEs as the above linear model ones, but here the CV RMSE is always higher than the Training RMSE so that is a plus, but still Lasso wins out</h3>

<a name="hyperparameter-dtree"></a>
##  Hyperparameter Tuning for Decision Tree

<b> Parameters tuned, min_samples_split, min_samples_leaf </b>

In [21]:
dtree_param_grid = {'min_samples_split': list(range(1,40)), 'min_samples_leaf': list(range(1,30))}
                      
grid_cv_dtree = GridSearchCV(DecisionTreeRegressor(), dtree_param_grid, n_jobs=-1,
                                       scoring='neg_root_mean_squared_error', verbose=2, cv=10, return_train_score=True)

for atm_name in atm_names:
    print("\nFor", atm_name)
    model_training_hyperparam_per_atm(atm_name, new_data, categorical_features_list, 
                                      grid_cv_dtree, tree_model=True)


For Big Street ATM
Fitting 10 folds for each of 1131 candidates, totalling 11310 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 1012 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 2636 tasks      | elapsed:   16.2s
[Parallel(n_jobs=-1)]: Done 4900 tasks      | elapsed:   27.2s
[Parallel(n_jobs=-1)]: Done 7820 tasks      | elapsed:   40.0s
[Parallel(n_jobs=-1)]: Done 11310 out of 11310 | elapsed:   53.7s finished


Best Parameters:
 {'min_samples_leaf': 6, 'min_samples_split': 16}

Best CV RMSE: 116438.93309299198
Best Training RMSE: 107746.12340204648
Test RMSE: 120993.33125770367
Year_2015: 0.504 +/- 0.034
Year_2014: 0.465 +/- 0.033
Year_2016: 0.434 +/- 0.032
Year_2013: 0.266 +/- 0.025
Year_2017: 0.205 +/- 0.016
Year_2012: 0.060 +/- 0.006
Day_12  : 0.017 +/- 0.006
Day_11  : 0.016 +/- 0.006
Month_7 : 0.014 +/- 0.006
Month_9 : 0.013 +/- 0.003
Day_6   : 0.013 +/- 0.002
Day_8   : 0.009 +/- 0.002
Month_12: 0.007 +/- 0.003
Holiday Sequence_WHH: 0.007 +/- 0.003
Weekday_MONDAY: 0.001 +/- 0.000
------------------------------------------------------------------------


For Mount Road ATM
Fitting 10 folds for each of 1131 candidates, totalling 11310 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 128 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 1096 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 2720 tasks      | elapsed:   13.6s
[Parallel(n_jobs=-1)]: Done 4984 tasks      | elapsed:   23.1s
[Parallel(n_jobs=-1)]: Done 7904 tasks      | elapsed:   34.4s
[Parallel(n_jobs=-1)]: Done 11310 out of 11310 | elapsed:   47.0s finished


Best Parameters:
 {'min_samples_leaf': 28, 'min_samples_split': 3}

Best CV RMSE: 194810.5737775145
Best Training RMSE: 179365.7213999431
Test RMSE: 198427.35122008406
Year_2016: 0.320 +/- 0.031
Year_2017: 0.258 +/- 0.029
Weekday_SUNDAY: 0.137 +/- 0.017
Festival Religion_NH: 0.057 +/- 0.008
Year_2014: 0.048 +/- 0.007
Year_2012: 0.024 +/- 0.005
Holiday Sequence_WHH: 0.011 +/- 0.005
Year_2013: 0.006 +/- 0.002
------------------------------------------------------------------------


For Airport ATM
Fitting 10 folds for each of 1131 candidates, totalling 11310 fits


...

Best Parameters:
 {'min_samples_leaf': 27, 'min_samples_split': 2}

Best CV RMSE: 179934.7183106889
Best Training RMSE: 163787.7055553611
Test RMSE: 184799.67409541932
Year_2016: 0.165 +/- 0.026
Weekday_SATURDAY: 0.068 +/- 0.017
Year_2017: 0.067 +/- 0.023
Year_2013: 0.066 +/- 0.013
Year_2012: 0.039 +/- 0.007
Working Day_W: 0.019 +/- 0.009
Day_7   : 0.011 +/- 0.004
Holiday Sequence_WWH: 0.002 +/- 0.001
------------------------------------------------------------------------


For KK Nagar ATM
Fitting 10 folds for each of 1131 candidates, totalling 11310 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 100 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 1068 tasks      | elapsed:    6.1s
[Parallel(n_jobs=-1)]: Done 2692 tasks      | elapsed:   14.1s
[Parallel(n_jobs=-1)]: Done 4956 tasks      | elapsed:   24.1s
[Parallel(n_jobs=-1)]: Done 7876 tasks      | elapsed:   35.7s
[Parallel(n_jobs=-1)]: Done 11310 out of 11310 | elapsed:   48.7s finished


Best Parameters:
 {'min_samples_leaf': 6, 'min_samples_split': 39}

Best CV RMSE: 318032.9409083069
Best Training RMSE: 297035.4601708347
Test RMSE: 321600.5866065472
Year_2014: 0.311 +/- 0.027
Weekday_SUNDAY: 0.265 +/- 0.024
Year_2013: 0.254 +/- 0.026
Year_2012: 0.196 +/- 0.020
Year_2017: 0.056 +/- 0.009
Year_2015: 0.039 +/- 0.007
Year_2016: 0.036 +/- 0.005
Month_12: 0.023 +/- 0.007
Day_6   : 0.022 +/- 0.006
Day_2   : 0.021 +/- 0.006
Month_11: 0.017 +/- 0.004
Festival Religion_NH: 0.016 +/- 0.007
Month_9 : 0.015 +/- 0.003
Day_12  : 0.015 +/- 0.005
Month_4 : 0.010 +/- 0.005
Holiday Sequence_HHW: 0.009 +/- 0.002
Month_8 : 0.007 +/- 0.003
Festival Religion_H: 0.003 +/- 0.001
Weekday_SATURDAY: 0.002 +/- 0.001
------------------------------------------------------------------------


For Christ College ATM
Fitting 10 folds for each of 1131 candidates, totalling 11310 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 128 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 1096 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done 2720 tasks      | elapsed:   14.3s
[Parallel(n_jobs=-1)]: Done 4984 tasks      | elapsed:   24.1s
[Parallel(n_jobs=-1)]: Done 7904 tasks      | elapsed:   35.3s
[Parallel(n_jobs=-1)]: Done 11310 out of 11310 | elapsed:   47.8s finished


Best Parameters:
 {'min_samples_leaf': 23, 'min_samples_split': 2}

Best CV RMSE: 230428.17247433276
Best Training RMSE: 208079.9391330452
Test RMSE: 225924.4704592382
Year_2017: 0.347 +/- 0.022
Year_2014: 0.291 +/- 0.026
Weekday_SUNDAY: 0.201 +/- 0.034
Year_2016: 0.181 +/- 0.014
Year_2015: 0.142 +/- 0.009
Year_2013: 0.099 +/- 0.011
Weekday_SATURDAY: 0.028 +/- 0.009
Year_2012: 0.026 +/- 0.004
------------------------------------------------------------------------



<table align="left">
    <tr>
        <th> ATM </th>
        <th> Train RMSE </th>
        <th> CV RMSE </th>
        <th> Test RMSE </th>
    </tr>
    <tr>
        <td> Big Street ATM </td>
        <td> 107746 </td>
        <td> 116438 </td>
        <td> 120993 </td>
    </tr>
    <tr>
        <td> Mount Road ATM </td>
        <td> 179365 </td>
        <td> 194810 </td>
        <td> 198427 </td>
    </tr>
    <tr>
        <td> Airport ATM </td>
        <td> 163787 </td>
        <td> 179934 </td>
        <td> 184799 </td>
    </tr>
    <tr>
        <td> KK Nagar ATM </td>
        <td> 297035 </td>
        <td> 318032 </td>
        <td> 321600 </td>
    </tr>
    <tr>
        <td> Christ College ATM </td>
        <td> 208079 </td>
        <td> 230428 </td>
        <td> 225924 </td>
    </tr>
</table>

<h3> Not as good as the Lasso but still quite good results and Decision Tree doesn't show the weird behaviour of CV RMSE being greater than Training RMSE </h3>

<a name="hyperparameter-rforest"></a>

## Hyperparameter Tuning for Random Forest
<b> Parameters tuned, n_estimators, min_samples_split, min_samples_leaf </b>

In [ ]:
rforest_param_grid = {'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)], 
                      'min_samples_split': list(range(1,40)), 'min_samples_leaf': list(range(1,25))}
                      
random_cv_rforest = RandomizedSearchCV(RandomForestRegressor(), rforest_param_grid, n_iter=25, n_jobs=-1, 
                                      scoring='neg_root_mean_squared_error', verbose=2, cv=10, return_train_score=True)

for atm_name in atm_names:
    print("\nFor", atm_name)
    model_training_hyperparam_per_atm(atm_name, new_data, categorical_features_list, 
                                      random_cv_rforest, tree_model=True)


For Big Street ATM
Fitting 10 folds for each of 25 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  9.2min finished


Best Parameters:
 {'n_estimators': 400, 'min_samples_split': 33, 'min_samples_leaf': 3}

Best CV RMSE: 111101.46970646169
Best Training RMSE: 104250.25653971334
Test RMSE: 115292.68592106046
Year_2015: 0.564 +/- 0.034
Year_2016: 0.447 +/- 0.032
Year_2014: 0.420 +/- 0.028
Year_2013: 0.308 +/- 0.017
Year_2017: 0.213 +/- 0.020
Year_2012: 0.102 +/- 0.009
Month_11: 0.026 +/- 0.011
Day_6   : 0.021 +/- 0.006
Day_11  : 0.019 +/- 0.005
Day_9   : 0.015 +/- 0.007
Month_12: 0.012 +/- 0.002
Day_8   : 0.010 +/- 0.004
Day_12  : 0.010 +/- 0.003
Day_24  : 0.005 +/- 0.002
------------------------------------------------------------------------


For Mount Road ATM
Fitting 10 folds for each of 25 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   36.7s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  9.1min finished


Best Parameters:
 {'n_estimators': 800, 'min_samples_split': 10, 'min_samples_leaf': 2}

Best CV RMSE: 188791.78164250223
Best Training RMSE: 181766.1802498414
Test RMSE: 173421.09880541897
Year_2016: 0.445 +/- 0.042
Year_2017: 0.388 +/- 0.038
Weekday_SUNDAY: 0.125 +/- 0.020
Holiday Sequence_HHW: 0.051 +/- 0.007
Festival Religion_NH: 0.044 +/- 0.009
Year_2014: 0.038 +/- 0.011
Day_10  : 0.032 +/- 0.008
Day_4   : 0.029 +/- 0.007
Year_2012: 0.028 +/- 0.006
Year_2015: 0.027 +/- 0.004
Day_11  : 0.025 +/- 0.005
Day_7   : 0.020 +/- 0.006
Month_11: 0.017 +/- 0.005
Day_3   : 0.011 +/- 0.004
Day_5   : 0.011 +/- 0.003
Day_9   : 0.010 +/- 0.005
Day_2   : 0.009 +/- 0.003
Day_6   : 0.008 +/- 0.004
Year_2013: 0.005 +/- 0.001
Holiday Sequence_WHH: 0.005 +/- 0.002
Day_31  : 0.004 +/- 0.002
Holiday Sequence_WWH: 0.002 +/- 0.001
Day_29  : 0.001 +/- 0.001
Festival Religion_N: 0.001 +/- 0.000
------------------------------------------------------------------------


For Airport ATM
Fitting 10 folds for eac

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   53.2s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  7.2min finished


Best Parameters:
 {'n_estimators': 400, 'min_samples_split': 16, 'min_samples_leaf': 4}

Best CV RMSE: 175576.68225787647
Best Training RMSE: 164774.20429220123
Test RMSE: 170434.22891491957
Year_2016: 0.234 +/- 0.021
Year_2017: 0.147 +/- 0.019
Day_8   : 0.054 +/- 0.008
Year_2013: 0.048 +/- 0.010
Year_2012: 0.029 +/- 0.006
Month_9 : 0.024 +/- 0.007
Year_2014: 0.022 +/- 0.005
Weekday_SATURDAY: 0.019 +/- 0.008
Working Day_W: 0.017 +/- 0.007
Month_11: 0.010 +/- 0.004
Day_5   : 0.007 +/- 0.002
Day_3   : 0.006 +/- 0.002
Day_6   : 0.005 +/- 0.002
Month_8 : 0.004 +/- 0.002
Month_4 : 0.004 +/- 0.002
Day_9   : 0.003 +/- 0.001
Day_25  : 0.003 +/- 0.001
Day_23  : 0.001 +/- 0.000
Day_24  : 0.001 +/- 0.000
------------------------------------------------------------------------


For KK Nagar ATM
Fitting 10 folds for each of 25 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  8.2min finished


Best Parameters:
 {'n_estimators': 2000, 'min_samples_split': 37, 'min_samples_leaf': 2}

Best CV RMSE: 293746.10272635485
Best Training RMSE: 291098.4424834337
Test RMSE: 315413.6836060787
Year_2014: 0.332 +/- 0.029
Year_2013: 0.289 +/- 0.028
Weekday_SUNDAY: 0.245 +/- 0.027
Year_2012: 0.213 +/- 0.022
Year_2017: 0.048 +/- 0.008
Year_2015: 0.039 +/- 0.005
Day_11  : 0.028 +/- 0.007
Month_12: 0.026 +/- 0.006
Day_6   : 0.024 +/- 0.006
Day_4   : 0.018 +/- 0.007
Day_9   : 0.015 +/- 0.003
Year_2016: 0.012 +/- 0.006
Month_9 : 0.012 +/- 0.003
Day_31  : 0.009 +/- 0.002
Day_2   : 0.009 +/- 0.004
Day_5   : 0.007 +/- 0.002
Holiday Sequence_HHW: 0.007 +/- 0.003
Month_7 : 0.006 +/- 0.002
Month_6 : 0.003 +/- 0.001
------------------------------------------------------------------------


For Christ College ATM
Fitting 10 folds for each of 25 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   55.2s


<table align="left">
    <tr>
        <th> ATM </th>
        <th> Train RMSE </th>
        <th> CV RMSE </th>
        <th> Test RMSE </th>
    </tr>
    <tr>
        <td> Big Street ATM </td>
        <td> 104250 </td>
        <td> 111101 </td>
        <td> 115292 </td>
    </tr>
    <tr>
        <td> Mount Road ATM </td>
        <td> 181766 </td>
        <td> 188791 </td>
        <td> 173421 </td>
    </tr>
    <tr>
        <td> Airport ATM </td>
        <td> 164774 </td>
        <td> 175576 </td>
        <td> 170434 </td>
    </tr>
    <tr>
        <td> KK Nagar ATM </td>
        <td> 291098 </td>
        <td> 293746 </td>
        <td> 315413 </td>
    </tr>
    <tr>
        <td> Christ College ATM </td>
        <td> 207308 </td>
        <td> 210016 </td>
        <td> 210450 </td>
    </tr>
</table>

<h3> Outperforms Lasso for all ATMs except Christ College but that's Random Search with just 25 iterations, increasing the iterations will definitely help find better results, so this means that Random Forest is the best model so far </h3>

## Conclusions from Hyperparameter Tuning

<ul>
    <li> Random Forest again performed the best </li>
    <li> In terms of important features, the important features varies a lot depending on which ATM is being modelled and which algorithm is being used. </li>
    <li> Because all we have are categorical columns, importance can only be found for single values of Categorical columns, so for example, we could have one model give "Weekday_SUNDAY" as important but all other Weekday values are not important for it, so it doesn't give a clear idea as to whether Weekday column is actually important for it or not </li>
    <li> Limiting work to single ATM which we have chosen as Big Street ATM will help narrow search down and it already is giving quite good results </li>

<a name="polynomial"></a>
## Polynomial Regression

<p> Generate a new feature matrix consisting of all polynomial combinations of the features with degree less than or equal to the specified degree. For example, if an input sample is two dimensional and of the form [a, b], the degree-2 polynomial features are [1, a, b, a^2, ab, b^2] </p>

<p> This is what the PolynomialFeatures() method does </p>

<b> Running time is very very slow, around 30 minutes for just training one ATM for one algorithm </b>

## Conclusions on Polynomial Regression
<p> All the models that Polynomial Regression was tried upon (except for Elasticnet), performed significantly worse than their regular counterparts and whichever ones did manage to equal the performance, they ended up highly overfitting the data </p>
<p> Possible reasons for this, the data consists of just categorical columns on which One-Hot Encoding was performed, so the only values present in each column is either a 0 or a 1, so it is quite possible that Polynomial Regression is not able to do much using Higher Order Features generated using such columns </p>
<p> The best training RMSE using regular hypertuned models was still above 1 lakhs, so definitely the models have to be more complex, so maybe once we have generated some numeric columns we can start working for Polynomial Regression </p>